In [0]:
%sql
-- Scripts for Agent's Tool:
-- Set catalog and schema
USE CATALOG workspace;
USE SCHEMA gold;

-- Drop old functions if they exist
DROP FUNCTION IF EXISTS workspace.gold.fn_monthly_sales;
DROP FUNCTION IF EXISTS workspace.gold.fn_monthly_revenue;
DROP FUNCTION IF EXISTS workspace.gold.fn_customer_purchases;
DROP FUNCTION IF EXISTS workspace.gold.fn_query_sales;

-- 1️⃣ Monthly Revenue Function
CREATE OR REPLACE FUNCTION workspace.gold.fn_monthly_revenue(
  start_date DATE,
  end_date   DATE
)
RETURNS TABLE (month DATE, revenue DOUBLE)
RETURN
  SELECT date_trunc('month', fs.order_date) AS month,
         SUM(fs.sales_amount)               AS revenue
  FROM workspace.gold.fact_sales fs
  WHERE fs.order_date BETWEEN start_date AND end_date
  GROUP BY 1
  ORDER BY 1;

-- 2️⃣ Customer Purchase Summary Function
CREATE OR REPLACE FUNCTION workspace.gold.fn_customer_purchases(
  customer_id STRING,
  start_date  DATE,
  end_date    DATE
)
RETURNS TABLE (
  order_number STRING,
  order_date DATE,
  product_name STRING,
  quantity INT,
  sales_amount DOUBLE
)
RETURN
  SELECT fs.order_number,
         fs.order_date,
         dp.product_name,
         fs.quantity,
         fs.sales_amount
  FROM workspace.gold.fact_sales fs
  JOIN workspace.gold.dim_products dp
    ON fs.product_key = dp.product_key
  JOIN workspace.gold.dim_customers dc
    ON fs.customer_key = dc.customer_key
  WHERE dc.customer_id = customer_id
    AND fs.order_date BETWEEN start_date AND end_date
  ORDER BY fs.order_date;

-- 3️⃣ Catch-All Sales Query Function
CREATE OR REPLACE FUNCTION workspace.gold.fn_query_sales()
RETURNS TABLE (
    order_number STRING,
    order_date DATE,
    customer_id STRING,
    customer_name STRING,
    product_id STRING,
    product_name STRING,
    quantity INT,
    sales_amount DOUBLE
)
RETURN
    SELECT 
        fs.order_number,
        fs.order_date,
        dc.customer_id,
        CONCAT(dc.first_name, ' ', dc.last_name) AS customer_name,
        dp.product_id,
        dp.product_name,
        fs.quantity,
        fs.sales_amount
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_customers dc
        ON fs.customer_key = dc.customer_key
    JOIN workspace.gold.dim_products dp
        ON fs.product_key = dp.product_key;

-- Create a year-filtered function

  CREATE OR REPLACE FUNCTION workspace.gold.fn_top_product_by_year(
    year INT
)
RETURNS TABLE (
    product_name STRING,
    total_quantity INT,
    total_revenue DOUBLE
)
RETURN
    SELECT 
        dp.product_name,
        SUM(fs.quantity) AS total_quantity,
        SUM(fs.sales_amount) AS total_revenue
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_products dp
        ON fs.product_key = dp.product_key
    WHERE YEAR(fs.order_date) = year
    GROUP BY dp.product_name
    ORDER BY total_quantity DESC
    LIMIT 1;

-- Top Product by Year
CREATE OR REPLACE FUNCTION workspace.gold.fn_top_products_by_year(
    year INT,
    top_n INT
)
RETURNS TABLE (
    product_name STRING,
    total_quantity INT,
    total_revenue DOUBLE
)
RETURN
    SELECT 
        dp.product_name,
        SUM(fs.quantity) AS total_quantity,
        SUM(fs.sales_amount) AS total_revenue
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_products dp
        ON fs.product_key = dp.product_key
    WHERE YEAR(fs.order_date) = year
    GROUP BY dp.product_name
    ORDER BY total_quantity DESC  -- or ORDER BY total_revenue DESC
    LIMIT top_n;

-- Top N Month by Revenue
CREATE OR REPLACE FUNCTION workspace.gold.fn_top_months_by_year(
    year INT,
    top_n INT
)
RETURNS TABLE (
    month DATE,
    revenue DOUBLE
)
RETURN
    SELECT 
        date_trunc('month', fs.order_date) AS month,
        SUM(fs.sales_amount) AS revenue
    FROM workspace.gold.fact_sales fs
    WHERE YEAR(fs.order_date) = year
    GROUP BY month
    ORDER BY revenue DESC
    LIMIT top_n;

-- Top customer by Revenue in a Year
CREATE OR REPLACE FUNCTION workspace.gold.fn_customer_by_year(
    year INT
)
RETURNS TABLE (
    customer_name STRING,
    total_revenue DOUBLE
)
RETURN
    SELECT 
        CONCAT(dc.first_name, ' ', dc.last_name) AS customer_name,
        SUM(fs.sales_amount) AS total_revenue
    FROM workspace.gold.fact_sales fs
    JOIN workspace.gold.dim_customers dc
        ON fs.customer_key = dc.customer_key
    WHERE YEAR(fs.order_date) = year
    GROUP BY dc.first_name, dc.last_name
    ORDER BY total_revenue DESC;